# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9663,-42.0244,26.10,85,95,8.08,BR,1707868347
1,1,hermanus,-34.4175,19.2361,15.47,88,6,1.90,ZA,1707868348
2,2,port-aux-francais,-49.3498,70.2200,7.85,90,100,9.51,TF,1707868348
3,3,kone,-21.0608,164.8619,30.75,62,12,2.31,NC,1707868348
4,4,tiksi,71.6366,128.8685,-24.43,99,100,1.97,RU,1707868349


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_width=800, frame_height=800, scale=1, size="Humidity", color="City")
# Display the map

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values

ideal_weather_df = ideal_weather_df.dropna()
# Display sample data

ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
121,121,papao,12.6919,-16.3792,25.06,83,0,3.09,SN,1707868393
125,125,porto novo,6.4991,2.6253,26.72,90,0,3.11,BJ,1707868395
162,162,xai-xai,-25.0445,33.6407,22.32,86,0,1.78,MZ,1707868410
232,232,acapulco de juarez,16.8680,-99.8940,26.78,65,0,2.38,MX,1707868435
289,289,pico truncado,-46.7992,-67.9577,25.78,14,0,1.39,AR,1707868458


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
121,papao,SN,12.6919,-16.3792,83,
125,porto novo,BJ,6.4991,2.6253,90,
162,xai-xai,MZ,-25.0445,33.6407,86,
232,acapulco de juarez,MX,16.8680,-99.8940,65,
289,pico truncado,AR,-46.7992,-67.9577,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(name_address.url)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
#     print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=130c1864f83d4b6cae25a2308385dc8d&filter=circle%3A-16.3792%2C12.6919%2C10000&bias=proximity%3A-16.3792%2C12.6919
papao - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=130c1864f83d4b6cae25a2308385dc8d&filter=circle%3A2.6253%2C6.4991%2C10000&bias=proximity%3A2.6253%2C6.4991
porto novo - nearest hotel: SODAS
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=130c1864f83d4b6cae25a2308385dc8d&filter=circle%3A33.6407%2C-25.0445%2C10000&bias=proximity%3A33.6407%2C-25.0445
xai-xai - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=130c1864f83d4b6cae25a2308385dc8d&filter=circle%3A-99.894%2C16.868%2C10000&bias=proximity%3A-99.894%2C16.868
acapulco de juarez - nearest hotel: Hotel del Valle
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=130c1864f83

,City,Country,Lat,Lng,Humidity,Hotel Name
121,papao,SN,12.6919,-16.3792,83,No hotel found
125,porto novo,BJ,6.4991,2.6253,90,SODAS
162,xai-xai,MZ,-25.0445,33.6407,86,No hotel found
232,acapulco de juarez,MX,16.8680,-99.8940,65,Hotel del Valle
289,pico truncado,AR,-46.7992,-67.9577,14,No hotel found
311,coachella,US,33.6795,-116.1763,22,Fantasy Springs Hotel
407,salalah,OM,17.0175,54.1011,73,Star Emirates Downtown
409,illapel,CL,-31.6327,-71.1683,64,Hotel Diaguitas
524,mahalapye,BW,-23.1116,26.8327,36,Maeto Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
mat_plot2 = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles='OSM', frame_width = 800, frame_height = 800, hover_cols = ["Hotel Name", "Country"], size="Humidity", color="City")
# Display the map
# YOUR CODE HERE
mat_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)